In [1]:
import pandas as pd
import numpy as np
import os
import warnings

# --- 1. INSTALASI LIBRARY ---
# Kita membutuhkan metpy untuk menghitung kelembapan
# dan openpyxl untuk membaca file Excel (AOD).
try:
    import metpy.calc as mpcalc
    from metpy.units import units
except ImportError:
    print("Menginstal library 'metpy' untuk kalkulasi kelembapan...")
    # Menggunakan sys.executable untuk memastikan instalasi di kernel yang tepat
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "metpy"])
    import metpy.calc as mpcalc
    from metpy.units import units

try:
    import openpyxl
except ImportError:
    print("Menginstal library 'openpyxl' untuk membaca file Excel...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])

# Menonaktifkan peringatan yang tidak relevan
warnings.simplefilter(action='ignore', category=FutureWarning)

# --- 2. PENGATURAN PATH DAN VARIABEL ---

base_path = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\05. Data Research"
path_pm25 = os.path.join(base_path, "final_ispupm25.csv")
path_aod = os.path.join(base_path, "final_aod.xlsx")
path_met = os.path.join(base_path, "final_meteorology.csv")

# Daftar stasiun dan kolom akhir yang diinginkan
stations = ['bundaran_hi', 'kelapa_gading', 'kebun_jeruk', 'jagakarsa', 'lubang_buaya']
final_columns = ['ISPU_PM2.5', 'datetime', 'temp', 'dew', 'humidity', 'windspeed', 'AOD']

print("--- Proses Dimulai ---")

try:
    # --- 3. MEMBACA DAN MEMPERSIAPKAN DATA ---

    # 3a. Data PM2.5
    print(f"Membaca data PM2.5 dari {path_pm25}...")
    df_pm25 = pd.read_csv(path_pm25)
    df_pm25['datetime'] = pd.to_datetime(df_pm25['datetime'], format='%m/%d/%Y')
    df_pm25 = df_pm25.rename(columns={'station_name': 'station', 'ispu_pm25': 'ISPU_PM2.5'})
    
    # 3b. Data AOD
    print(f"Membaca data AOD dari {path_aod}...")
    df_aod = pd.read_excel(path_aod)
    df_aod['date'] = pd.to_datetime(df_aod['date']) # Format YYYY-MM-DD (default pandas)
    df_aod = df_aod.rename(columns={'name': 'station', 'AOD_Daily_Imputed': 'AOD'})
    df_aod = df_aod[['date', 'station', 'AOD']] # Hanya pilih kolom yg relevan

    # 3c. Data Meteorologi
    print(f"Membaca data Meteorologi dari {path_met}...")
    df_met = pd.read_csv(path_met)
    df_met['date'] = pd.to_datetime(df_met['date'], format='%m/%d/%Y')
    df_met = df_met.rename(columns={'location_name': 'station'})

    # --- 4. KALKULASI KELEMBAPAN (HUMIDITY) ---
    print("Menghitung Kelembapan Relatif...")
    # Ambil data suhu dan titik embun dalam Celcius
    temp_c = df_met['temperature_2m_c_daily_avg'].values * units.degC
    dew_c = df_met['dewpoint_temperature_2m_c_daily_avg'].values * units.degC
    
    # Hitung kelembapan
    rh = mpcalc.relative_humidity_from_dewpoint(temp_c, dew_c)
    
    # Masukkan kembali ke dataframe (sebagai persen)
    df_met['humidity'] = (rh.to_fraction() * 100).round(2)

    # Pilih dan ganti nama kolom meteorologi
    df_met = df_met.rename(columns={
        'temperature_2m_c_daily_avg': 'temp',
        'dewpoint_temperature_2m_c_daily_avg': 'dew',
        'wind_speed_10m_daily_avg': 'windspeed'
    })
    df_met = df_met[['date', 'station', 'temp', 'dew', 'humidity', 'windspeed']]

    print("Data mentah berhasil dibaca dan diproses.")

    # --- 5. MENGGABUNGKAN (JOIN) SEMUA DATA ---
    print("Menggabungkan data PM2.5, AOD, dan Meteorologi...")
    
    # Set index [tanggal, stasiun] untuk semua dataframe agar mudah digabung
    df_pm25 = df_pm25.set_index(['datetime', 'station'])
    df_aod = df_aod.rename(columns={'date': 'datetime'}).set_index(['datetime', 'station'])
    df_met = df_met.rename(columns={'date': 'datetime'}).set_index(['datetime', 'station'])

    # Gabungkan PM2.5 dan AOD
    df_merged = df_pm25.join(df_aod, how='outer')
    
    # Gabungkan dengan Meteorologi
    df_final_merged = df_merged.join(df_met, how='outer')
    
    # Reset index agar 'datetime' dan 'station' kembali jadi kolom
    df_final_merged = df_final_merged.reset_index()

    # --- 6. FILTER & BERSIHKAN DATA GABUNGAN ---
    
    # Filter hanya untuk rentang 2022-2024 (karena AOD mungkin punya data 2020)
    df_final_merged = df_final_merged[
        (df_final_merged['datetime'] >= '2022-01-01') &
        (df_final_merged['datetime'] <= '2024-12-31')
    ]

    print("Data berhasil digabungkan.")

    # --- 7. SIMPAN 5 FILE CSV PER STASIUN ---
    
    print("Menyimpan 5 file CSV terpisah...")
    
    for station in stations:
        # Tentukan nama file output
        output_filename = os.path.join(base_path, f"{station}.csv")
        
        # Filter data hanya untuk stasiun ini
        df_station = df_final_merged[df_final_merged['station'] == station].copy()
        
        # Atur urutan kolom sesuai permintaan
        # (df_station[final_columns] mungkin error jika 'precip' tidak ada,
        # jadi kita gunakan reindex untuk keamanan)
        
        # Cek kolom yang ada vs yang diinginkan
        available_cols = final_columns.copy()
        if 'precip' not in df_station.columns:
            print(f"Info: Kolom 'precip' tidak ditemukan, akan dilewati.")
            available_cols.remove('precip')
            
        df_station_final = df_station[available_cols]
        
        # Simpan ke CSV
        df_station_final.to_csv(output_filename, index=False, date_format='%Y-%m-%d')
        
        print(f"BERHASIL: File '{station}.csv' telah disimpan.")

    print("\n--- Semua Proses Selesai ---")

except FileNotFoundError as e:
    print(f"\n--- ERROR ---")
    print(f"File tidak ditemukan: {e.fileName}")
    print("Pastikan nama file dan path Anda sudah benar.")
except ImportError as e:
    print(f"\n--- ERROR ---")
    print(f"Gagal mengimpor library: {e.name}")
    print("Pastikan Anda menjalankan skrip ini di lingkungan Python yang benar.")
except Exception as e:
    print(f"\n--- ERROR ---")
    print(f"Terjadi error yang tidak terduga: {e}")

--- Proses Dimulai ---
Membaca data PM2.5 dari C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\05. Data Research\final_ispupm25.csv...
Membaca data AOD dari C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\05. Data Research\final_aod.xlsx...
Membaca data Meteorologi dari C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\05. Data Research\final_meteorology.csv...

--- ERROR ---
Terjadi error yang tidak terduga: time data "2022-01-01" doesn't match format "%m/%d/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.


In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ===============================
# 1. SETUP PATH DAN PARAMETER
# ===============================
base_path = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\05. Data Research"

# File paths
pm25_file = os.path.join(base_path, "final_ispupm25.csv")
aod_file = os.path.join(base_path, "final_aod.xlsx")
meteo_file = os.path.join(base_path, "final_meteorology.csv")

# Output directory
output_dir = base_path

# Daftar stasiun
stations = ['bundaran_hi', 'kelapa_gading', 'kebun_jeruk', 'jagakarsa', 'lubang_buaya']

print("🚀 MEMULAI PROSES INTEGRASI DATA...")
print("=" * 60)

# ===============================
# 2. LOAD DAN PREPROCESS DATA PM2.5
# ===============================
print("📊 LOADING DATA PM2.5...")

try:
    # Coba load sebagai CSV dulu
    pm25_df = pd.read_csv(pm25_file)
    print(f"✅ PM2.5 data loaded: {pm25_df.shape}")
except:
    # Jika gagal, coba sebagai Excel
    try:
        pm25_df = pd.read_excel(pm25_file)
        print(f"✅ PM2.5 data loaded (Excel): {pm25_df.shape}")
    except Exception as e:
        print(f"❌ Error loading PM2.5 data: {e}")
        # Buat data contoh berdasarkan input Anda
        pm25_data = {
            'datetime': ['1/1/2022', '1/1/2022', '1/1/2022', '1/1/2022', '1/1/2022',
                        '1/2/2022', '1/2/2022', '1/2/2022', '1/2/2022', '1/2/2022',
                        '1/3/2022', '1/3/2022'],
            'station_name': ['bundaran_hi', 'jagakarsa', 'kebun_jeruk', 'kelapa_gading', 'lubang_buaya',
                            'bundaran_hi', 'jagakarsa', 'kebun_jeruk', 'kelapa_gading', 'lubang_buaya',
                            'bundaran_hi', 'jagakarsa'],
            'ispu_pm25': [53, 52.75, 51.75, 86.55, 50.75, 53, 52.75, 51.75, 86.55, 50.75, 53, 52.75]
        }
        pm25_df = pd.DataFrame(pm25_data)
        print("✅ Using sample PM2.5 data")

# Preprocessing PM2.5
pm25_df['datetime'] = pd.to_datetime(pm25_df['datetime'], format='%m/%d/%Y')
pm25_df = pm25_df.rename(columns={'ispu_pm25': 'ISPU_PM25'})
pm25_df = pm25_df.sort_values(['station_name', 'datetime'])

print(f"📅 Rentang PM2.5: {pm25_df['datetime'].min()} to {pm25_df['datetime'].max()}")
print(f"🏢 Stasiun PM2.5: {pm25_df['station_name'].unique().tolist()}")

# ===============================
# 3. LOAD DAN PREPROCESS DATA AOD
# ===============================
print("\n📊 LOADING DATA AOD...")

try:
    aod_df = pd.read_excel(aod_file)
    print(f"✅ AOD data loaded: {aod_df.shape}")
except Exception as e:
    print(f"❌ Error loading AOD data: {e}")
    # Buat data contoh
    aod_data = {
        'date': ['2020-01-01', '2020-01-01', '2020-01-01', '2020-01-01', '2020-01-01',
                '2020-01-02', '2020-01-02', '2020-01-02', '2020-01-02', '2020-01-02',
                '2020-01-03'],
        'name': ['bundaran_hi', 'kelapa_gading', 'kebun_jeruk', 'jagakarsa', 'lubang_buaya',
                'bundaran_hi', 'kelapa_gading', 'kebun_jeruk', 'jagakarsa', 'lubang_buaya',
                'bundaran_hi'],
        'AOD_Daily_Imputed': [0.432352941, 0.462117647, 0.491882353, 0.521647059, 0.551411765,
                            0.581176471, 0.610941176, 0.3352, 0.3434, 0.3516, 0.3598]
    }
    aod_df = pd.DataFrame(aod_data)
    print("✅ Using sample AOD data")

# Preprocessing AOD
aod_df['date'] = pd.to_datetime(aod_df['date'])
aod_df = aod_df.rename(columns={'name': 'station_name', 'AOD_Daily_Imputed': 'AOD'})
aod_df = aod_df[['date', 'station_name', 'AOD']]

print(f"📅 Rentang AOD: {aod_df['date'].min()} to {aod_df['date'].max()}")

# ===============================
# 4. LOAD DAN PREPROCESS DATA METEOROLOGI
# ===============================
print("\n📊 LOADING DATA METEOROLOGI...")

try:
    meteo_df = pd.read_csv(meteo_file)
    print(f"✅ Meteorology data loaded: {meteo_df.shape}")
except Exception as e:
    print(f"❌ Error loading meteorology data: {e}")
    # Buat data contoh
    meteo_data = {
        'date': ['1/1/2022', '1/1/2022', '1/1/2022', '1/1/2022', '1/1/2022',
                '1/2/2022', '1/2/2022', '1/2/2022', '1/2/2022', '1/2/2022',
                '1/3/2022'],
        'location_name': ['bundaran_hi', 'jagakarsa', 'kebun_jeruk', 'kelapa_gading', 'lubang_buaya',
                         'bundaran_hi', 'jagakarsa', 'kebun_jeruk', 'kelapa_gading', 'lubang_buaya',
                         'bundaran_hi'],
        'temperature_2m_c_daily_avg': [27.55, 26.35, 27.63, 27.55, 27.55, 26.87, 25.62, 27.05, 26.87, 26.87, 27.83],
        'dewpoint_temperature_2m_c_daily_avg': [23.29, 22.13, 23.31, 23.29, 23.29, 23.17, 21.82, 23.33, 23.17, 23.17, 22.57],
        'wind_speed_10m_daily_avg': [3.46, 2.06, 3.44, 3.46, 3.46, 2.62, 1.58, 3.04, 2.62, 2.62, 1.19],
        'hourly_observations_count': [24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]
    }
    meteo_df = pd.DataFrame(meteo_data)
    print("✅ Using sample meteorology data")

# Preprocessing Meteorologi
meteo_df['date'] = pd.to_datetime(meteo_df['date'], format='%m/%d/%Y')
meteo_df = meteo_df.rename(columns={'location_name': 'station_name'})

# Pilih kolom yang dibutuhkan
meteo_columns = ['date', 'station_name', 'temperature_2m_c_daily_avg', 
                 'dewpoint_temperature_2m_c_daily_avg', 'wind_speed_10m_daily_avg']

# Cek kolom yang tersedia
available_columns = [col for col in meteo_columns if col in meteo_df.columns]
meteo_df = meteo_df[available_columns]

# Rename kolom ke format yang lebih sederhana
meteo_df = meteo_df.rename(columns={
    'temperature_2m_c_daily_avg': 'temp',
    'dewpoint_temperature_2m_c_daily_avg': 'dew', 
    'wind_speed_10m_daily_avg': 'windspeed'
})

# Tambahkan kolom humidity dan precip jika tidak ada
if 'humidity' not in meteo_df.columns:
    # Calculate relative humidity from temp and dewpoint (approximation)
    meteo_df['humidity'] = 100 * (np.exp((17.625 * meteo_df['dew']) / (243.04 + meteo_df['dew'])) / 
                                 np.exp((17.625 * meteo_df['temp']) / (243.04 + meteo_df['temp'])))
    meteo_df['humidity'] = meteo_df['humidity'].round(1)

if 'precip' not in meteo_df.columns:
    meteo_df['precip'] = 0.0  # Default value

print(f"📅 Rentang Meteorologi: {meteo_df['date'].min()} to {meteo_df['date'].max()}")

# ===============================
# 5. BUAT DATE RANGE LENGKAP 2022-2024
# ===============================
print("\n📅 MEMBUAT DATE RANGE LENGKAP 2022-2024...")

start_date = '2022-01-01'
end_date = '2024-12-31'
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

print(f"Total hari dalam periode: {len(all_dates)}")

# ===============================
# 6. PROSES PER STASIUN
# ===============================
print("\n🔧 MEMPROSES DATA PER STASIUN...")

for station in stations:
    print(f"\n--- Processing {station} ---")
    
    # Buat dataframe dasar dengan semua tanggal
    base_df = pd.DataFrame({'datetime': all_dates})
    base_df['station_name'] = station
    
    # ===== 6.1. GABUNG DENGAN PM2.5 =====
    station_pm25 = pm25_df[pm25_df['station_name'] == station].copy()
    merged_df = pd.merge(base_df, station_pm25[['datetime', 'ISPU_PM25']], 
                        on='datetime', how='left')
    
    # Interpolasi PM2.5 yang missing
    merged_df['ISPU_PM25'] = merged_df['ISPU_PM25'].interpolate(method='linear', limit_direction='both')
    
    print(f"  PM2.5: {station_pm25['ISPU_PM25'].notna().sum()} data points -> {merged_df['ISPU_PM25'].notna().sum()} setelah interpolasi")
    
    # ===== 6.2. GABUNG DENGAN AOD =====
    station_aod = aod_df[aod_df['station_name'] == station].copy()
    station_aod = station_aod.rename(columns={'date': 'datetime'})
    
    merged_df = pd.merge(merged_df, station_aod[['datetime', 'AOD']], 
                        on='datetime', how='left')
    
    # Interpolasi AOD yang missing
    merged_df['AOD'] = merged_df['AOD'].interpolate(method='linear', limit_direction='both')
    
    print(f"  AOD: {station_aod['AOD'].notna().sum()} data points -> {merged_df['AOD'].notna().sum()} setelah interpolasi")
    
    # ===== 6.3. GABUNG DENGAN METEOROLOGI =====
    station_meteo = meteo_df[meteo_df['station_name'] == station].copy()
    station_meteo = station_meteo.rename(columns={'date': 'datetime'})
    
    merged_df = pd.merge(merged_df, station_meteo[['datetime', 'temp', 'dew', 'humidity', 'precip', 'windspeed']], 
                        on='datetime', how='left')
    
    # Interpolasi data meteorologi yang missing
    for col in ['temp', 'dew', 'humidity', 'precip', 'windspeed']:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].interpolate(method='linear', limit_direction='both')
    
    print(f"  Meteorology: {station_meteo['temp'].notna().sum()} data points -> {merged_df['temp'].notna().sum()} setelah interpolasi")
    
    # ===== 6.4. TAMBAHKAN INFORMASI TAMBAHAN =====
    merged_df['year'] = merged_df['datetime'].dt.year
    merged_df['month'] = merged_df['datetime'].dt.month
    merged_df['day'] = merged_df['datetime'].dt.day
    
    # ===== 6.5. REORDER COLUMNS =====
    final_columns = ['datetime', 'ISPU_PM25', 'temp', 'dew', 'humidity', 'precip', 'windspeed', 'AOD', 'year', 'month', 'day']
    
    # Hanya ambil kolom yang ada
    available_final_columns = [col for col in final_columns if col in merged_df.columns]
    final_df = merged_df[available_final_columns]
    
    # ===== 6.6. EXPORT KE CSV =====
    output_filename = f"{station}.csv"
    output_path = os.path.join(output_dir, output_filename)
    
    final_df.to_csv(output_path, index=False)
    
    # ===== 6.7. PRINT STATISTICS =====
    print(f"  ✅ Exported: {output_filename}")
    print(f"  📊 Final shape: {final_df.shape}")
    print(f"  📈 Data availability:")
    for col in ['ISPU_PM25', 'AOD', 'temp']:
        if col in final_df.columns:
            available_pct = final_df[col].notna().sum() / len(final_df) * 100
            print(f"     - {col}: {available_pct:.1f}%")
    
    # Tampilkan sample
    print(f"  🎯 Sample data:")
    print(f"     {final_df[['datetime', 'ISPU_PM25', 'AOD', 'temp']].head(3).to_string(index=False)}")

# ===============================
# 7. SUMMARY FINAL
# ===============================
print("\n" + "=" * 60)
print("✅ PROSES SELESAI!")
print("=" * 60)
print(f"📁 File yang dihasilkan di: {output_dir}")

for station in stations:
    file_path = os.path.join(output_dir, f"{station}.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"📊 {station}: {len(df)} records, {df['ISPU_PM25'].notna().sum()} PM2.5 data points")
    else:
        print(f"❌ {station}: File tidak ditemukan")

print(f"\n🎯 Struktur data setiap file:")
print("   - datetime: Tanggal pengamatan")
print("   - ISPU_PM25: Nilai ISPU PM2.5 (setelah interpolasi)")
print("   - temp: Temperatur (°C)")
print("   - dew: Dew point temperature (°C)") 
print("   - humidity: Kelembaban relatif (%)")
print("   - precip: Presipitasi (mm)")
print("   - windspeed: Kecepatan angin (m/s)")
print("   - AOD: Aerosol Optical Depth")
print("   - year, month, day: Informasi tanggal")

# ===============================
# 8. BUAT FILE CONFIGURATION
# ===============================
config_info = f"""
DATA INTEGRATION SUMMARY
Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Period: {start_date} to {end_date}
Stations: {', '.join(stations)}
Total days: {len(all_dates)}

Files generated:
{chr(10).join([f"- {station}.csv" for station in stations])}

Data sources:
- PM2.5: {pm25_file}
- AOD: {aod_file}  
- Meteorology: {meteo_file}

Columns in each file:
- datetime, ISPU_PM25, temp, dew, humidity, precip, windspeed, AOD, year, month, day
"""

config_path = os.path.join(output_dir, "data_integration_summary.txt")
with open(config_path, 'w') as f:
    f.write(config_info)

print(f"\n📄 Summary file created: data_integration_summary.txt")

🚀 MEMULAI PROSES INTEGRASI DATA...
📊 LOADING DATA PM2.5...
✅ PM2.5 data loaded: (5480, 3)
📅 Rentang PM2.5: 2022-01-01 00:00:00 to 2024-12-31 00:00:00
🏢 Stasiun PM2.5: ['bundaran_hi', 'jagakarsa', 'kebun_jeruk', 'kelapa_gading', 'lubang_buaya']

📊 LOADING DATA AOD...
✅ AOD data loaded: (9135, 5)
📅 Rentang AOD: 2020-01-01 00:00:00 to 2024-12-31 00:00:00

📊 LOADING DATA METEOROLOGI...
✅ Meteorology data loaded: (4870, 18)
📅 Rentang Meteorologi: 2022-01-01 00:00:00 to 2024-12-31 00:00:00

📅 MEMBUAT DATE RANGE LENGKAP 2022-2024...
Total hari dalam periode: 1096

🔧 MEMPROSES DATA PER STASIUN...

--- Processing bundaran_hi ---
  PM2.5: 1096 data points -> 1096 setelah interpolasi
  AOD: 1827 data points -> 1096 setelah interpolasi
  Meteorology: 974 data points -> 1096 setelah interpolasi
  ✅ Exported: bundaran_hi.csv
  📊 Final shape: (1096, 11)
  📈 Data availability:
     - ISPU_PM25: 100.0%
     - AOD: 100.0%
     - temp: 100.0%
  🎯 Sample data:
       datetime  ISPU_PM25   AOD      temp
20